# Scraping the Courts and Tribunals Judiciary Website to collect Prevent Future Death (PFD) reports 

In [1]:
from requests import get
from requests import ConnectionError
from bs4 import BeautifulSoup
import re
from time import sleep
from time import time
import csv
import pandas as pd

from tqdm.auto import tqdm
    
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
 
def get_url(url):
    response = get(url, verify = False)
    html = response.content
    soup = BeautifulSoup(html, "html.parser")
    return soup

def retries(record_url, tries=3):
    for i in range(tries):
        try:
            soup = get_url(record_url)
            return soup
        except (ConnectionError, SSLError):
            if i < tries - 1:
                sleep(2)
                continue
            else:
                return 'Con error'

# Scraper starts here - last run on Friday, 03 April 2021, 5.40 pm.

In [2]:
#Your second value in "range" will be one more than the number of pages that exist on the wesbite
pages = list(range(1,353))

#This loops through all the pages to get the URLs to individual records
page_string = 'https://www.judiciary.uk/subject/prevention-of-future-deaths/page/{}/'
record_urls = []
for page in tqdm(pages):
    soup = get_url(page_string.format(str(page)))
    h5s = soup.find_all('h5', {'class': 'entry-title'})
    for h5 in h5s:
        record_urls.append(h5.a.get('href'))

Here we check how many records (i.e. cases) were pulled from the urls & the first and last case

In [3]:
len(record_urls)

3516

In [4]:
record_urls[0]

'https://www.judiciary.uk/publications/joan-coley/'

In [5]:
record_urls[-1]

'https://www.judiciary.uk/publications/phillip-pratt/'

Here is my second loop. This will go through the lists of URLs I just created above to visit each individual record and pull out and store the text data (info on the decreased/case) and the PDF URL I will use later

In [6]:
def error_details(e_dict, record_count, record_url, details):
    e_dict['index'] = record_count
    e_dict['url'] = record_url
    e_dict['reason'] = details
    return e_dict

In [7]:
reg_exp = re.compile(r"’s\s|s\s|'s\s")
text_cats = ['Date of report', 'Ref', 'Deceased name', 'Coroner name', 'Coroner Area', 'Category', "This report is being sent to"]
#First, I create two lists, one for the PDFs and one for the text data
record_text = []
pdf_urls = []
ref_list = []
#I want to loop through each URL & pull out the death information and pdf link for downloading
error_catching = []

record_count = 0
for record_url in tqdm(record_urls):
    try:
        error_dict = {}
        #Calling the retries function
        soup = retries(record_url, tries=5)
        
        if soup == 'Con error':
            print(f"{record_url} could not connect")
            error_catching.append(error_details(error_dict, record_count, record_url, 'Connection Error'))
            record_count +=1
            continue

        #This gets all the text fields from the website to work with
        death_info = soup.find('div', {'class':'entry-content'}).find_all('p')
        
        if not death_info:
            print(f"{record_url} produced no data")
            error_catching.append(error_details(error_dict, record_count, record_url, 'No Text Loaded'))
            record_count +=1
            continue
            
        #Our dictionary that will hold all of the text information that we will eventually append to "record_text"
        blankdict = {}
        
        #This is to handle 1 annoying record with messed up html tags
        if record_url == 'https://www.judiciary.uk/publications/roadsafety/':
            strong = death_info[0].find_all('strong')
            heads = ['date_of_report', 'ref', 'deceased_name', 'coroner_name', 'coroner_area', 'category']
            for st, h in zip(strong,heads):
                blankdict[h] = st.next_sibling.replace(':','').replace('Ref','').strip()
        #And another record with wonky html
        elif record_url == 'https://www.judiciary.uk/publications/helen-sheath/':
            brs = death_info[0].text.split('\n')
            vals = []
            for b in brs:
                vals.append(b.split(':'))
            for v in vals:
                if v[0] == "Coroners name":
                    alt = "coroner_name"
                    blankdict[alt] = v[1].strip().replace('\n','')
                elif v[0] == "Coroners Area":
                    alt = "coroner_area"
                    blankdict[alt] = v[1].strip().replace('\n','')
                else:
                    blankdict[v[0].strip().replace(' ','_').lower()] = v[1].strip().replace('\n','')
        else:        
            #looping through all of the text categories for handling
            for p in death_info:
                #This checks for blank fields and if there is nothing, it skips it
                if p.text.strip() == '':
                    pass
                #This checks for our "Normal" case in which a colon exists and the category is one of the ones we 
                #pre-specified above in the "text_cats" list
                #We also need to account here for one strange record for "Rebecca Evans" which has a weird text error
                #That we manually correct for
                elif ':' in p.text and p.text.split(':')[0] in text_cats and not 'Rebecca-EvansR.pdf' in p.text:
                    #Simply assigning the key and value from strings on either side of the colon, making everything 
                    #lower case and replacing spaces with underscores and also removing any stray semi-colons
                    text_list = p.text.split(':')
                    blankdict[text_list[0].strip().replace(' ','_').lower()] = text_list[1].strip().replace('\n','').replace('\xa0','')

                elif 'Rebecca-EvansR.pdf' in p.text:
                    #This deals with that singular odd record that currently exists as of 8 Nov 2019
                    blankdict['category'] = p.text.split(':')[1].strip().replace('\n','')
                    
                elif ':' not in p.text:
                    #If the string doesn't have a colon, we can't split on it so have to get it into dictionary format
                    #Using an alternate method that counts the length of the thing
                    if any(x in p.text for x in text_cats):
                        t = [x for x in text_cats if x in p.text][0]
                        l = len(t)
                        blankdict[t.replace(' ','_').lower()] = p.text[l+1:].replace('\n','').replace('\xa0','')
                    elif 'Coroners Area' in p.text:
                        blankdict['coroner_area'] = p.text[13:].strip().replace('\n','').replace('\xa0','')
                    else:
                        print("Something we haven't accounted for has happened")

                elif p.text.strip().count(":") == 2:
                    #This corrects for one odd record in which there are 2 colons but should generalize to fix it for
                    #any time this could happen, so long as it happens in the same way
                    text_list = p.text.split(':')
                    new_string = text_list[0] + text_list[1]
                    new_name = re.sub(reg_exp, ' ', new_string).strip()
                    blankdict[new_name.replace(' ','_').lower()] = text_list[2].strip().replace('\n','').replace('\xa0','')

                elif ':' in p.text and p.text.split(':')[0] not in text_cats:
                    #Some field names are in the form of "name_of_decesased" or "name_of_coroner" or are plural/
                    #possessive so this smashes those into our preferred naming formats
                    if 'Name of' in p.text:
                        all_text = p.text.split(':')
                        key_name = all_text[0].split(' ')
                        blankdict[key_name[2].strip() + '_name'] = all_text[-1].strip()
                    else:    
                        new_name = re.sub(reg_exp, ' ', p.text)
                        text_list = new_name.split(':')
                        blankdict[text_list[0].strip().replace(' ','_').lower()] = text_list[1].strip().replace('\n','').replace('\xa0','')
        blankdict['url'] = record_url
        
        #A small little check for duplicated ref names
        try:
            if not blankdict['ref']:
                pass
            elif blankdict['ref'] in ref_list:
                blankdict['ref'] = blankdict['ref'] + 'A'
            ref_list.append(blankdict['ref'])
        except KeyError:
            blankdict['ref'] = ''
            
        #This appends the final dict to the list
        record_text.append(blankdict)
        
        #this is a seperate process to get the PDF URLs (no matter how many there are) and adds them to their own list   
        urls = soup.find_all('li', {'class':'pdf'})
        pdf_list = []
        for url in urls:
            pdf_list.append(url.findNext('a').get('href'))
        pdf_urls.append(pdf_list)
        
        record_count += 1
        
    except Exception as e:
        import sys
        error_desc = f"{str(e)} occurred for {record_url} when trying to work with {p}"
        print(error_desc)
        error_catching.append(error_details(error_dict, record_count, record_url, error_desc))
        
        #Saving this in case we don't like the error catching.
        #import sys
        #raise type(e)(str(e) + '\n' + 'Error for Record: {}, Field: {}'.format(record_url, p)).with_traceback(sys.exc_info()[2])

Something we haven't accounted for has happened
Something we haven't accounted for has happened
https://www.judiciary.uk/publications/railwayrelateddeaths/ produced no data
https://www.judiciary.uk/publications/service-personnel-deaths/ produced no data
https://www.judiciary.uk/publications/product/ produced no data
https://www.judiciary.uk/publications/policerelateddeaths/ produced no data
https://www.judiciary.uk/publications/carehomehealth/ produced no data
https://www.judiciary.uk/publications/statecustodydeath/ produced no data
https://www.judiciary.uk/publications/hospitaldeath/ produced no data
https://www.judiciary.uk/publications/drugsalcoholmedication/ produced no data
https://www.judiciary.uk/publications/commhealthcareemergencyservices/ produced no data



Here is the third loop to save the PDFs using the deceased Ref as the file name

In [8]:
#Any errors should print out above, but you can also check the error_catching dict
#Here we just turn it into a dataframe quickly to easily view

error_df = pd.DataFrame(error_catching)
error_df

,index,reason,url
0,3445,No Text Loaded,https://www.judiciary.uk/publications/railwayr...
1,3446,No Text Loaded,https://www.judiciary.uk/publications/service-...
2,3447,No Text Loaded,https://www.judiciary.uk/publications/product/
3,3448,No Text Loaded,https://www.judiciary.uk/publications/policere...
4,3449,No Text Loaded,https://www.judiciary.uk/publications/carehome...
5,3450,No Text Loaded,https://www.judiciary.uk/publications/statecus...
6,3453,No Text Loaded,https://www.judiciary.uk/publications/hospital...
7,3511,No Text Loaded,https://www.judiciary.uk/publications/drugsalc...
8,3512,No Text Loaded,https://www.judiciary.uk/publications/commheal...


In [10]:
def save_file(path_string, name_string):
    with open(path_string.format(name_string), 'wb') as d:
        d.write(myfile.content)

save_path = '/Users/georgiarichards/Desktop/Python/PFDs opioids/All_PDFs6/{}.pdf'

potential_names = ['ref', 'deceased_name', 'date_of_report']

record_count = 0
#This is the final scrape to actually get the URLs and change the name (when possible) to the refs
for r_t, p_u in zip(tqdm(record_text), pdf_urls):
    if not p_u:
        #If there is no pdf at all, we skip it.
        continue
    else:
        #All this does is gets the PDF and downloads it and names it after the reg
        #It looks scary and complicated but all it is doing is varying the name in the case of multiple PDFs
        #Or naming it for the deceased person if there is no Ref value
        #If there is a pdf but no ref or deceased name, this will throw an error and we can adjust.
        try:
            counter = 0
            if len(p_u) > 1:
                for p in p_u:
                    if counter == 0:
                        myfile = get(p)
                        named = False
                        for x in potential_names:
                            try:
                                if r_t[x]:
                                    save_file(save_path, r_t[x])
                                    counter +=1
                                    named = True
                                    break
                                else:
                                    continue
                            except KeyError:
                                continue
                        if not named:       
                            save_file(save_path, 'check_record_{}'.format(record_count))
                            counter +=1

                    else:
                        myfile = get(p)
                        named = False
                        for x in potential_names:
                            try:
                                if r_t[x]:
                                    save_file(save_path, r_t[x] + '_{}'.format(counter))
                                    counter +=1
                                    named = True
                                    break
                                else:
                                    continue
                            except KeyError:
                                continue
                        if not named:
                            save_file(save_path, 'check_record_{}_{}'.format(record_count, counter))
                            counter +=1
                                    
            else:
                myfile = get(p_u[0])
                named = False
                for x in potential_names:
                    try:
                        if r_t[x]:
                            save_file(save_path, r_t[x])
                            named = True
                            break
                        else:
                            continue
                    except KeyError:
                        continue
                if not named:       
                    save_file(save_path, 'check_record_{}'.format(record_count))
            
            record_count += 1
        
        except Exception as e:
            import sys
            if r_t['ref']:
                raise type(e)(str(e) + '\n' + 'Error for Record: {}'.format(r_t['ref'])).with_traceback(sys.exc_info()[2])
            else:
                raise type(e)(str(e) + '\n' + 'Error for Record Number: {}'.format(record_count)).with_traceback(sys.exc_info()[2])

This is my final step that puts the text data (info on the deceased/case) into a csv file & adds the date it was pulled

In [12]:
from datetime import date

headers = ['date_of_report', 'date_of_reports', 'ref', 'deceased_name', 'deceased_names', 'coroner_name', 'coroner_area', 'category', 'this_report_is_being_sent_to', 'these_report_are_being_sent_to', 'url']

with open('death_info_{}.csv'.format(date.today()), 'w', newline='', encoding='utf-8') as deaths_csv:
    writer = csv.DictWriter(deaths_csv, fieldnames=headers)
    writer.writeheader()
    for record in record_text:
        if record == {}:
            pass
        else:
            writer.writerow(record)

This is an addition few steps to check what differences there are from the August 2020 records 

In [13]:
import os

pdfs5 = os.listdir('All_PDFs5')
pdfs6 = os.listdir('All_PDFs6')

new_not_old = set(pdfs6).difference(pdfs5)

new_not_old_list = list(new_not_old)
new_not_old_list.sort()
new_not_old_list

['2013-0265A.pdf',
 '2013-0265A_1.pdf',
 '2013-0347A.pdf',
 '2013-0347A_1.pdf',
 '2014-0301A.pdf',
 '2014-0301A_1.pdf',
 '2014-0341_2.pdf',
 '2014-0424A.pdf',
 '2014-0462A.pdf',
 '2014-0520A.pdf',
 '2014-0520A_1.pdf',
 '2014-0549A.pdf',
 '2014-0549A_1.pdf',
 '2014-0556A.pdf',
 '2014-0556A_1.pdf',
 '2015-0020A.pdf',
 '2015-0020A_1.pdf',
 '2015-0049A.pdf',
 '2015-0100A.pdf',
 '2015-0100A_1.pdf',
 '2015-0102A.pdf',
 '2015-0102A_1.pdf',
 '2015-0102A_2.pdf',
 '2015-0228A.pdf',
 '2015-0228A_1.pdf',
 '2015-0243A.pdf',
 '2015-0261A.pdf',
 '2015-0338A.pdf',
 '2015-0369A.pdf',
 '2015-0369A_1.pdf',
 '2015-0418A.pdf',
 '2016 – 0146A.pdf',
 '2016 – 0172A.pdf',
 '2016 – 0172A_1.pdf',
 '2016 – 0396A.pdf',
 '2016 – 0396A_1.pdf',
 '2016-0001A.pdf',
 '2016-0001A_1.pdf',
 '2016-0001A_2.pdf',
 '2016-0056A.pdf',
 '2016-0219A.pdf',
 '2016-0219A_1.pdf',
 '2016-0219A_2.pdf',
 '2016-0436A.pdf',
 '2017-0157A.pdf',
 '2017-0210A.pdf',
 '2018-0231A.pdf',
 '2018-0231A_1.pdf',
 '2018-0231A_2.pdf',
 '2019-0119_1.pdf'

In [14]:
len(new_not_old_list)

658

In [15]:
apr21 = pd.read_csv('death_info_2021-04-02.csv')
sep20 = pd.read_csv('death_info_2020-09-19.csv')

In [17]:
cols = list(sep20.columns)
merged = apr21.merge(sep20, on=cols, how='left', indicator=True)

In [18]:
l_only = merged[merged['_merge'] == 'left_only']

In [19]:
len(l_only)

266

In [20]:
l_only

,date_of_report,date_of_reports,ref,deceased_name,deceased_names,coroner_name,coroner_area,category,this_report_is_being_sent_to,these_report_are_being_sent_to,url,_merge
0,31 March 2021,NaN,2021-0093,Joan Coley,NaN,Louise Hunt,Birmingham and Solihull,Hospital Death (Clinical Procedures and medica...,Sandwell and West Birmingham Hospitals NHS Tru...,NaN,https://www.judiciary.uk/publications/joan-coley/,left_only
1,30 October 2020,NaN,2021-0092,Michael Robert Collins,NaN,Nadia Persaud,East London,Hospital Death (Clinical Procedures and medica...,Royal London Hospital,NaN,https://www.judiciary.uk/publications/michael-...,left_only
2,8 December 2020,NaN,2021-0091,Ann Stillwell,NaN,Graeme Irvine,East London,Care Home Health related deaths,Department of Health and Social Care and Haver...,NaN,https://www.judiciary.uk/publications/ann-stil...,left_only
3,26 March 2021,NaN,2021-0090,Rachel Johnston,NaN,David Reid,Worcestershire,Care Home Health related deaths,Holmleigh Care Homes Ltd and Care Quality Comm...,NaN,https://www.judiciary.uk/publications/rachel-j...,left_only
4,29 March 2021,NaN,2021-0089,Raymond Powell,NaN,James Bennett,Birmingham and Solihull,Care Home Health related deaths,Cole Valley Care Ltd,NaN,https://www.judiciary.uk/publications/raymond-...,left_only
5,1 December 2020,NaN,2021-0088,Brandon-Robert Collins-Hayward,NaN,Rachael Griffin,Dorset,Hospital Death (Clinical Procedures and medica...,Royal College of Obstetricians and Gynaecologi...,NaN,https://www.judiciary.uk/publications/brandon-...,left_only
6,28 March 2021,NaN,2021-0087,Nicholas Rousseau,NaN,Dr Sean Cummings,Milton Keynes,Hospital Death (Clinical Procedures and medica...,Milton Keynes University Hospital,NaN,https://www.judiciary.uk/publications/nicholas...,left_only
7,10 December 2020,NaN,2021-0086,Rory Attwood,NaN,Caroline Saunders,Gwent,"Mental Health related deaths | Alcohol, drug a...",Aneurin Bevan University Health Board,NaN,https://www.judiciary.uk/publications/rory-att...,left_only
8,26 March 2021,NaN,2021-0085,Clara Freeman,NaN,Stephen Hugh Glossop Covell,Plymouth Torbay and South Devon,Care Home Health related deaths,Hart Care Nursing and Residential Home,NaN,https://www.judiciary.uk/publications/clara-fr...,left_only
9,26 March 2021,NaN,2021-0084,Lee Marsden,NaN,Matthew Cox,Manchester North,Road (Highways Safety) related deaths |Police ...,Highways England and North West Motorway Polic...,NaN,https://www.judiciary.uk/publications/lee-mars...,left_only


In [21]:
l_only.to_csv(r'death_info_newapr21.csv')

Data processing for website 

In [22]:
apr_names = pd.read_csv('death_info_2021-04-02.csv')
apr_names.head()

,date_of_report,date_of_reports,ref,deceased_name,deceased_names,coroner_name,coroner_area,category,this_report_is_being_sent_to,these_report_are_being_sent_to,url
0,31 March 2021,NaN,2021-0093,Joan Coley,NaN,Louise Hunt,Birmingham and Solihull,Hospital Death (Clinical Procedures and medica...,Sandwell and West Birmingham Hospitals NHS Tru...,NaN,https://www.judiciary.uk/publications/joan-coley/
1,30 October 2020,NaN,2021-0092,Michael Robert Collins,NaN,Nadia Persaud,East London,Hospital Death (Clinical Procedures and medica...,Royal London Hospital,NaN,https://www.judiciary.uk/publications/michael-...
2,8 December 2020,NaN,2021-0091,Ann Stillwell,NaN,Graeme Irvine,East London,Care Home Health related deaths,Department of Health and Social Care and Haver...,NaN,https://www.judiciary.uk/publications/ann-stil...
3,26 March 2021,NaN,2021-0090,Rachel Johnston,NaN,David Reid,Worcestershire,Care Home Health related deaths,Holmleigh Care Homes Ltd and Care Quality Comm...,NaN,https://www.judiciary.uk/publications/rachel-j...
4,29 March 2021,NaN,2021-0089,Raymond Powell,NaN,James Bennett,Birmingham and Solihull,Care Home Health related deaths,Cole Valley Care Ltd,NaN,https://www.judiciary.uk/publications/raymond-...


In [23]:
apr_names['deceased_name'] = apr_names['deceased_name'].fillna("")

In [24]:
apr_names['deceased_name'] = apr_names['deceased_name'].apply(lambda x: ''.join(i[0] for i in x.split()))

In [25]:
apr_names['deceased_name'] = apr_names['deceased_name'].str.replace('\W', '')

In [26]:
apr_names.head()

,date_of_report,date_of_reports,ref,deceased_name,deceased_names,coroner_name,coroner_area,category,this_report_is_being_sent_to,these_report_are_being_sent_to,url
0,31 March 2021,NaN,2021-0093,JC,NaN,Louise Hunt,Birmingham and Solihull,Hospital Death (Clinical Procedures and medica...,Sandwell and West Birmingham Hospitals NHS Tru...,NaN,https://www.judiciary.uk/publications/joan-coley/
1,30 October 2020,NaN,2021-0092,MRC,NaN,Nadia Persaud,East London,Hospital Death (Clinical Procedures and medica...,Royal London Hospital,NaN,https://www.judiciary.uk/publications/michael-...
2,8 December 2020,NaN,2021-0091,AS,NaN,Graeme Irvine,East London,Care Home Health related deaths,Department of Health and Social Care and Haver...,NaN,https://www.judiciary.uk/publications/ann-stil...
3,26 March 2021,NaN,2021-0090,RJ,NaN,David Reid,Worcestershire,Care Home Health related deaths,Holmleigh Care Homes Ltd and Care Quality Comm...,NaN,https://www.judiciary.uk/publications/rachel-j...
4,29 March 2021,NaN,2021-0089,RP,NaN,James Bennett,Birmingham and Solihull,Care Home Health related deaths,Cole Valley Care Ltd,NaN,https://www.judiciary.uk/publications/raymond-...


In [27]:
apr_names.to_csv('death_info_2021-04-02_processed.csv')